# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import Workspace, Experiment,Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig

import os
import joblib

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment

In [ ]:
experiment_name = 'automl-experiment'
project_folder = './automl-experiment'

experiment = Experiment(ws, experiment_name)
experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
dataset=Dataset.get_by_name(ws,name="Credit-Card-Churners")
run = exp.start_logging()

In [ ]:
currDir=os.getcwd()
print(currDir)
os.listdir(currDir)

## Config Compute Cluster

In [ ]:
compute_name = "compute-aml-cluster"

# Check if the compute target exists
try:
    compute_aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisionibng_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=5)
    compute_aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

compute_aml_cluster.wait_for_completion(show_output=True)

In [ ]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

all_data = pd.concat([x,y], axis = 1)
all_data.head()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(all_data, test_size=0.2,random_state=0)
# columns=x_train.columns
print(x_train.head())
print(x_test.head())
x_train.to_csv('training/train_data.csv')
x_test.to_csv('training/test_data.csv')

# x_train.to_csv(path_or_buf='training/train_data.csv', columns=columns, header=True, index=False) 
# x_test.to_csv(path_or_buf='training/test_data.csv', columns=columns, header=True, index=False)

In [ ]:
# get the datastore to upload prepared data
datastore = ws.get_default_datastore()
print(datastore)

# upload the local file from src_dir to the target_path in datastore
datastore.upload(src_dir='training/', target_path='data/')
# datastore.upload_files(['trainset.csv'])

# create a dataset referencing the cloud location
train_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ('data/train_data.csv'))])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 10,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations"=45,
    "max_cores_per_iteration"=-1,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_aml_cluster,
                             task = "classification",
                             primary_metric='accuracy',
                             training_data=dataset,
                             label_column_name="Attrition_Flag",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
remote_run

In [ ]:
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_automl_best_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()
print('Best Run ID',best_run.id)


In [ ]:
best_run

In [ ]:
fitted_automl_best_model

In [ ]:
# Metric in best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)
    
print('\nAccuracy of Best run',best_run_metrics['accuracy'],sep='\n')
print(best_run)

model_name = best_run.properties['model_name']
model_name

In [ ]:
#TODO: Save the best model

os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service